In [1]:
from kashgari.corpus import SMP2018ECDTCorpus
import torch
import re
import os
from tensorflow import *
from transformers import *
import pytorch_pretrained_bert
import numpy as np
import re
import pandas as pd

╭─────────────────────────────────────────────────────────────────────────╮
│ ◎ ○ ○ ░░░░░░░░░░░░░░░░░░░░░  Important Message  ░░░░░░░░░░░░░░░░░░░░░░░░│
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│              We renamed again for consistency and clarity.              │
│                   From now on, it is all `kashgari`.                    │
│  Changelog: https://github.com/BrikerMan/Kashgari/releases/tag/v1.0.0   │
│                                                                         │
│         | Backend          | pypi version   | desc           |          │
│         | ---------------- | -------------- | -------------- |          │
│         | TensorFlow 2.x   | kashgari 2.x.x | coming soon    |          │
│         | TensorFlow 1.14+ | kashgari 1.x.x |                |          │
│         | Keras            | kashgari 0.x.x | legacy version |          │
│           

In [80]:
import tensorflow as tf
tf.test.is_gpu_available()

False

In [26]:
example = pd.read_table("E:\\model\\train.txt")
example.columns = ['id', 'texta', 'textb', 'label']
example.head()
trainset = example.sample(frac=0.7, replace=False, weights=None, random_state=0, axis=0)
trainset.head()
valiset = example[~example.index.isin(trainset.index)]
valiset.head()
trainset.to_csv('E:\\model\\trainset.txt', sep='\t', index=False)
valiset.to_csv('E:\\model\\valiset.txt', sep='\t', index=False)

In [13]:
#BERT trial
examples_train=[]
examples_label=[]
with open("E:\\model\\trainset.txt") as reader:
    while True:
        line = reader.readline()
        line
        if not line:
            break
        text_a=line.split('\t')[1]
        text_b=line.split('\t')[2]
        label = line.strip('\n').split('\t')[3]
        examples_label.append(label)
        examples_train.append(text_a)
        examples_train.append(text_b)

In [38]:
with open("E:\\model\\trainset.txt") as reader:
    line=reader.readline()
    print(line)

In [14]:
examples_train

['Two girls in red soccer uniforms running after a soccer ball.',
 'Two girls are playing a game of soccer.',
 'Two people eat and converse outdoors.',
 'There are individuals eating outside.',
 'An older man in safety orange holds a piece of wood next to another older man with gray hair.',
 'Two men walk past each other.',
 'Asian women in traditional garb are walking in a parade.',
 'Asian women in a parade',
 'Two young boys sit outside in the daylight hours wearing Winnie the Pooh and animal costumes.',
 'the boys are sitting insided at night',
 'Young boy in blue shirt on spiral staircase in a room with many books.',
 'The shirt is large.',
 'Four young children eating lunch outside.',
 'The children are inside.',
 'Two people jousting with lances and shields on horses while people watch from under a tent and outside of the tent.',
 'There is a jousting competition going on.',
 'A man and a woman are holding hands as they walk along a city sidewalk.',
 'A married couple are holdin

In [27]:
import kashgari
from kashgari.embeddings import BERTEmbedding

bert_embedding = BERTEmbedding('E:\\model',
                               task=kashgari.CLASSIFICATION,
                               sequence_length=128)
bert_embedding.processor.add_bos_eos = False
tokenizer = bert_embedding.tokenizer

In [32]:
bert_tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")

In [20]:
len(examples_train)

19598

In [33]:
i=0
train_x=[]
while i<=9798:
    train_x.append(bert_tokenizer.tokenize(examples_train[i]))
    i=i+1

In [47]:
#BERT trial
i=0
train_x=[]
while i<=19597:
    token_a=bert_tokenizer.tokenize(examples_train[i])
    token_b=bert_tokenizer.tokenize(examples_train[i+1])
    token=token_a+["[SEP]"]+token_b
    train_x.append(token)
    i=i+2

In [41]:
len(train_x)

9799

In [38]:
train_y=examples_label

In [39]:
train_y

['entailment',
 'entailment',
 'contradiction',
 'entailment',
 'contradiction',
 'neutral',
 'contradiction',
 'neutral',
 'neutral',
 'entailment',
 'neutral',
 'neutral',
 'contradiction',
 'neutral',
 'neutral',
 'entailment',
 'neutral',
 'neutral',
 'contradiction',
 'contradiction',
 'neutral',
 'entailment',
 'neutral',
 'entailment',
 'neutral',
 'contradiction',
 'entailment',
 'entailment',
 'entailment',
 'entailment',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'entailment',
 'neutral',
 'contradiction',
 'neutral',
 'neutral',
 'neutral',
 'entailment',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'entailment',
 'entailment',
 'entailment',
 'contradiction',
 'neutral',
 'contradiction',
 'neutral',
 'contradiction',
 'contradiction',
 'neutral',
 'neutral',
 'contradiction',
 'entailment',
 'entailment',
 'entailment',
 'contradiction',
 'contradiction',
 'neutral',
 'contradiction',
 'contradiction',
 'neutral',
 'contradiction',
 'entailment',
 'entailment',
 

In [40]:
len(train_y)

9799

In [57]:
examples_valid=[]
valid_label=[]
with open("E:\\model\\valiset.txt") as reader:
    while True:
        line = reader.readline()
        line
        if not line:
            break
        text= line.split('\t')[1]+'SEP '+line.split('\t')[2]
        text_a=line.split('\t')[1]
        text_b=line.split('\t')[2]
        label = line.strip('\n').split('\t')[3]
        valid_label.append(label)
        examples_valid.append(text)

In [42]:
#BERT trial
examples_valid=[]
valid_label=[]
with open("E:\\model\\valiset.txt") as reader:
    while True:
        line = reader.readline()
        line
        if not line:
            break
        text_a=line.split('\t')[1]
        text_b=line.split('\t')[2]
        label = line.strip('\n').split('\t')[3]
        valid_label.append(label)
        examples_valid.append(text_a)
        examples_valid.append(text_b)

In [43]:
len(examples_valid)

8400

In [59]:
i=0
valid_x=[]
while i<=4199:
    valid_x.append(bert_tokenizer.tokenize(examples_valid[i]))
    i=i+1

In [46]:
i=0
valid_x=[]
while i<=8399:
    token_a=bert_tokenizer.tokenize(examples_valid[i])
    token_b=bert_tokenizer.tokenize(examples_valid[i+1])
    token=token_a+["[SEP]"]+token_b
    valid_x.append(token)
    i=i+2

In [45]:
valid_y=valid_label

In [50]:
len(valid_y)

4200

In [59]:
i=0
valid_x=[]
while i<=4199:
    valid_x.append(bert_tokenizer.tokenize(examples_valid[i]))
    i=i+1

In [52]:
examples_test=[]
with open("E:\\model\\test_matched.txt") as reader:
    while True:
        line = reader.readline()
        line
        if not line:
            break
        text_a= line.split('\t')[1]
        text_b=line.split('\t')[2]
        examples_test.append(text_a)
        examples_test.append(text_b)

In [53]:
len(examples_test)

10000

In [76]:
i=0
test_x=[]
while i<=4999:
    test_x.append(bert_tokenizer.tokenize(examples_test[i]))
    i=i+1

In [54]:
#BERT trial
i=0
test_x=[]
while i<=9999:
    token_a=bert_tokenizer.tokenize(examples_test[i])
    token_b=bert_tokenizer.tokenize(examples_test[i+1])
    token=token_a+["[SEP]"]+token_b
    test_x.append(token)
    i=i+2

In [55]:
test_x

[['a',
  'man',
  'in',
  'a',
  'green',
  'apron',
  'roast',
  '##s',
  'a',
  'pig',
  'over',
  'coal',
  '##s',
  '.',
  '`',
  '[SEP]',
  'the',
  'man',
  'roast',
  '##s',
  'the',
  'pig',
  '.'],
 ['a',
  'man',
  'in',
  'a',
  'green',
  'apron',
  'roast',
  '##s',
  'a',
  'pig',
  'over',
  'coal',
  '##s',
  '.',
  '`',
  '[SEP]',
  'the',
  'woman',
  'is',
  'roast',
  '##ing',
  'a',
  'duck',
  '.'],
 ['a',
  'man',
  'in',
  'a',
  'green',
  'apron',
  'roast',
  '##s',
  'a',
  'pig',
  'over',
  'coal',
  '##s',
  '.',
  '`',
  '[SEP]',
  'the',
  'hawaii',
  '##n',
  'man',
  'roast',
  '##s',
  'a',
  'pig',
  'over',
  'coal',
  '##s',
  '.'],
 ['an',
  'asian',
  'man',
  'cooking',
  'meat',
  'over',
  'an',
  'open',
  'grill',
  '.',
  '[SEP]',
  'the',
  'guy',
  'cooks',
  'meat'],
 ['an',
  'asian',
  'man',
  'cooking',
  'meat',
  'over',
  'an',
  'open',
  'grill',
  '.',
  '[SEP]',
  'the',
  'guy',
  'cooks',
  'meat',
  'for',
  'the',
  'fami

In [70]:
import kashgari
from kashgari.tasks.classification import BiLSTM_Model

import logging
logging.basicConfig(level='DEBUG')

hyper = BiLSTM_Model.get_default_hyper_parameters()
print(hyper)
# {'layer_bi_lstm': {'units': 128, 'return_sequences': False}, 'layer_dense': {'activation': 'softmax'}}
)
hyper['layer_bi_lstm']['units'] =48
#hyper['layer_bi_lstm']['return_sequences']=True

model = BiLSTM_Model(hyper_parameters=hyper

In [81]:
kashgari.config.use_cudnn_cell = True

In [71]:
model.fit(train_x, train_y, valid_x, valid_y)

Model: "model_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 39)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 39, 100)           435300    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 96)                57216     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 291       
Total params: 492,807
Trainable params: 492,807
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
154/154 [==============================] - ETA: 15:42 - loss: 1.0982 - acc: 0.32 - ETA: 7:57 - loss: 1.0988 - acc: 0.3047 - ETA: 5:21 - loss: 1.0985 - acc: 0.307 - ETA: 4:03 - loss: 1.0982 - acc: 0.328 - ETA: 3:17 - loss: 1.1005 - acc: 0.30

154/154 [==============================] - ETA: 16s - loss: 0.9786 - acc: 0.46 - ETA: 15s - loss: 0.9616 - acc: 0.54 - ETA: 15s - loss: 0.9517 - acc: 0.55 - ETA: 14s - loss: 0.9553 - acc: 0.55 - ETA: 14s - loss: 0.9686 - acc: 0.53 - ETA: 14s - loss: 0.9584 - acc: 0.54 - ETA: 14s - loss: 0.9590 - acc: 0.54 - ETA: 14s - loss: 0.9497 - acc: 0.54 - ETA: 14s - loss: 0.9409 - acc: 0.55 - ETA: 14s - loss: 0.9329 - acc: 0.56 - ETA: 14s - loss: 0.9285 - acc: 0.57 - ETA: 13s - loss: 0.9228 - acc: 0.57 - ETA: 13s - loss: 0.9205 - acc: 0.57 - ETA: 13s - loss: 0.9213 - acc: 0.57 - ETA: 13s - loss: 0.9135 - acc: 0.57 - ETA: 13s - loss: 0.9163 - acc: 0.57 - ETA: 13s - loss: 0.9134 - acc: 0.57 - ETA: 13s - loss: 0.9076 - acc: 0.57 - ETA: 13s - loss: 0.9043 - acc: 0.57 - ETA: 13s - loss: 0.8973 - acc: 0.58 - ETA: 13s - loss: 0.8936 - acc: 0.58 - ETA: 12s - loss: 0.8896 - acc: 0.58 - ETA: 12s - loss: 0.8880 - acc: 0.58 - ETA: 12s - loss: 0.8895 - acc: 0.58 - ETA: 12s - loss: 0.8893 - acc: 0.58 - ETA: 12

154/154 [==============================] - ETA: 15s - loss: 0.6350 - acc: 0.76 - ETA: 15s - loss: 0.6321 - acc: 0.75 - ETA: 15s - loss: 0.6256 - acc: 0.74 - ETA: 15s - loss: 0.6252 - acc: 0.74 - ETA: 15s - loss: 0.6462 - acc: 0.74 - ETA: 15s - loss: 0.6445 - acc: 0.74 - ETA: 15s - loss: 0.6538 - acc: 0.73 - ETA: 15s - loss: 0.6689 - acc: 0.72 - ETA: 15s - loss: 0.6739 - acc: 0.71 - ETA: 15s - loss: 0.6637 - acc: 0.72 - ETA: 15s - loss: 0.6576 - acc: 0.72 - ETA: 14s - loss: 0.6502 - acc: 0.72 - ETA: 14s - loss: 0.6449 - acc: 0.73 - ETA: 14s - loss: 0.6424 - acc: 0.73 - ETA: 15s - loss: 0.6521 - acc: 0.72 - ETA: 15s - loss: 0.6410 - acc: 0.73 - ETA: 15s - loss: 0.6384 - acc: 0.73 - ETA: 15s - loss: 0.6394 - acc: 0.73 - ETA: 15s - loss: 0.6383 - acc: 0.73 - ETA: 15s - loss: 0.6395 - acc: 0.73 - ETA: 15s - loss: 0.6366 - acc: 0.73 - ETA: 15s - loss: 0.6360 - acc: 0.73 - ETA: 15s - loss: 0.6365 - acc: 0.73 - ETA: 15s - loss: 0.6329 - acc: 0.73 - ETA: 14s - loss: 0.6256 - acc: 0.74 - ETA: 14

In [67]:
model.save('saved_classification_model')
loaded_model = kashgari.utils.load_model('saved_classification_model')

In [78]:
result=loaded_model.predict(test_x)

In [77]:
import kashgari
from kashgari.tasks.classification import BiGRU_Model
from kashgari.embeddings import BERTEmbedding
from kashgari.tasks.classification import BiLSTM_Model

import logging
logging.basicConfig(level='DEBUG')
bert_embed = BERTEmbedding('E:\\model',
                           task=kashgari.CLASSIFICATION,
                           trainable=True,
                           sequence_length=125)
bert_embed

<<class 'kashgari.embeddings.bert_embedding.BERTEmbedding'> seq_len: 125>

In [73]:
?BiGRU_Model

In [84]:
model = BiGRU_Model(bert_embed)
model.fit(train_x, train_y, valid_x, valid_y,batch_size=12)

Model: "model_49"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 125)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 125)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 125, 768), ( 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 125, 768)     1536        Input-Segment[0][0]              
___________________________________________________________________________________________

Encoder-9-MultiHeadSelfAttentio (None, 125, 768)     2362368     Encoder-8-FeedForward-Norm[0][0] 
__________________________________________________________________________________________________
Encoder-9-MultiHeadSelfAttentio (None, 125, 768)     0           Encoder-9-MultiHeadSelfAttention[
__________________________________________________________________________________________________
Encoder-9-MultiHeadSelfAttentio (None, 125, 768)     0           Encoder-8-FeedForward-Norm[0][0] 
                                                                 Encoder-9-MultiHeadSelfAttention-
__________________________________________________________________________________________________
Encoder-9-MultiHeadSelfAttentio (None, 125, 768)     1536        Encoder-9-MultiHeadSelfAttention-
__________________________________________________________________________________________________
Encoder-9-FeedForward (FeedForw (None, 125, 768)     4722432     Encoder-9-MultiHeadSelfAttention-
__________

KeyboardInterrupt: 

In [69]:
model.save('saved_bert_classification_model')
loaded_model = kashgari.utils.load_model('saved_bert_classification_model')

In [77]:
result=loaded_model.predict(test_x)

In [78]:
import pandas  as pd
names=['id','label']
test=pd.DataFrame(columns=names,data=result)
test
test.to_csv('e:/result1.csv',encoding='gbk')